In [1]:
import os

In [2]:
%pwd

'c:\\Users\\admin\\Desktop\\Induction_Furnace_Carbon_Sulphur_Response\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\admin\\Desktop\\Induction_Furnace_Carbon_Sulphur_Response'

In [5]:
# data validation 

import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/Mild_Steel.csv")
data.head()

,LADDLE_TEMP,CASTING_TEMP,AVG_POWER_PER_HEAT_KW,POWER_CONSUMED_KWH,FURNACE_COOLING_PRESSURE,CONVERTOR_COOLING_PRESSURE,SPONGE_IRON_MT,SCRAP_MT,PIG_IRON_KG,CPC_KG,SiMn_KG,AluminiumBar_KG,Carbon(C%),Manganese(Mn%),Silicon(Si%),Sulphur(S%),Phosphorous(P%),MT_PRODUCTION_PER_HEAT
0,1670,1622,6000,8700.0,0.18,0.21,3.5,12.5,700,100,12.5,2.0,0.15,0.6,0.7,0.024,0.018,13.5
1,1675,1618,5200,13260.0,0.18,0.22,4.5,12.0,500,60,9.5,2.0,0.18,0.7,0.8,0.022,0.017,12.5
2,1670,1625,5500,11550.0,0.18,0.23,4.5,12.0,500,60,10.0,2.0,0.17,0.7,0.8,0.021,0.017,12.3
3,1675,1628,6000,8700.0,0.18,0.23,10.5,4.5,1500,100,12.5,2.0,0.17,0.5,0.7,0.015,0.016,12.0
4,1670,1626,6000,9000.0,0.19,0.24,10.5,4.5,1500,100,12.5,2.0,0.18,0.6,0.8,0.016,0.018,12.5


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10830 entries, 0 to 10829
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   LADDLE_TEMP                 10830 non-null  int64  
 1   CASTING_TEMP                10830 non-null  int64  
 2   AVG_POWER_PER_HEAT_KW       10830 non-null  int64  
 3   POWER_CONSUMED_KWH          10830 non-null  float64
 4   FURNACE_COOLING_PRESSURE    10830 non-null  float64
 5   CONVERTOR_COOLING_PRESSURE  10830 non-null  float64
 6   SPONGE_IRON_MT              10830 non-null  float64
 7   SCRAP_MT                    10830 non-null  float64
 8   PIG_IRON_KG                 10830 non-null  int64  
 9   CPC_KG                      10830 non-null  int64  
 10  SiMn_KG                     10830 non-null  float64
 11  AluminiumBar_KG             10830 non-null  float64
 12  Carbon(C%)                  10830 non-null  float64
 13  Manganese(Mn%)              108

In [8]:
data.isnull().sum()

LADDLE_TEMP                   0
CASTING_TEMP                  0
AVG_POWER_PER_HEAT_KW         0
POWER_CONSUMED_KWH            0
FURNACE_COOLING_PRESSURE      0
CONVERTOR_COOLING_PRESSURE    0
SPONGE_IRON_MT                0
SCRAP_MT                      0
PIG_IRON_KG                   0
CPC_KG                        0
SiMn_KG                       0
AluminiumBar_KG               0
Carbon(C%)                    0
Manganese(Mn%)                0
Silicon(Si%)                  0
Sulphur(S%)                   0
Phosphorous(P%)               0
MT_PRODUCTION_PER_HEAT        0
dtype: int64

In [9]:
data.shape

(10830, 18)

In [10]:
# prepare entity 

from dataclasses import dataclass
from pathlib import Path 


@dataclass(frozen = True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    valid_data_dir: Path
    all_schema: dict

In [11]:
from src.Mlflow_Project.constants import *
from src.Mlflow_Project.utils.utility import FileOperations

In [16]:
# configuration manager class , will read all yaml files 

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = FileOperations.read_yaml(config_filepath)
        self.params = FileOperations.read_yaml(params_filepath)
        self.schema = FileOperations.read_yaml(schema_filepath)

        FileOperations.create_directories([self.config.artifacts_root])

    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS


        FileOperations.create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            valid_data_dir = config.valid_data_dir,
            all_schema=schema,
        
    )


        return data_validation_config

In [13]:
# Building componets 

import os 
from src.Mlflow_Project.__init__ import logger 

In [1]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config 


    # validating all columns 

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None 

            data = pd.read_csv(self.config.valid_data_dir)
            all_cols = list(data.columns)


            all_schema = self.config.all_schema.keys()


            for col in all_cols:
                if col not in all_schema:
                    validation_status = False 
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status: {validation_status}")

                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status: {validation_status}")


            return validation_status
        
        except Exception as e:
            print("An error occurred:", e)

        

NameError: name 'DataValidationConfig' is not defined

In [20]:
# initializing my pipeline 

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    print("An error occurred:", e)

[2023-11-09 13:18:49,691: INFO: yaml file: config\config.yaml loaded successfully]
[2023-11-09 13:18:49,694: INFO: yaml file: params.yaml loaded successfully]
[2023-11-09 13:18:49,705: INFO: yaml file: schema.yaml loaded successfully]
[2023-11-09 13:18:49,708: INFO: created directory at: artifacts]
[2023-11-09 13:18:49,710: INFO: created directory at: artifacts/data_validation]
